# TIAGo tutorial

---

## Reference
 - [TIAGo Tutorial(ROS公式)](http://wiki.ros.org/Robots/TIAGo/Tutorials)
 - TIAGO_lang_en_GB_handbook-1.29.0-ferrum.pdf(マニュアル, 280ページからチュートリアルに言及)

## Environment
 - OS:Ubuntu 18.04.5 LTS by using WSL2 on windows10
 - ROS distribution: melodic

---

## 1. Tutorial Installation
1. Installing Ubuntu with ROS + TIAGO    
  - rosdp packegeはmelodicインストール時に同時にインストールしてくれなくなったので別でインストールが必要 
      - 現時点ではそのまま'sudo rosdp init'でいきなり初期化しているがこれだと存在しないとエラーが出る 
      > [rosdpをインストールする手順が変わったようです](https://memoteki.net/archives/2778)
2. Install Tiago Tutorial Docker
    - Dockerで環境ごと持ってくるパターン(やってない)
3. Testing the simulation
    - ~/tiago_public_ws/で実行(以降のコマンドも特に指定がなければそう)
    - launch command:`roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=_value_`
      - _value_ = steelとするとデフォルトの手
      - _value_ = titaniumとすると指が5本ある手(Hey 5)
---

## 2. Control
1. Teleoperating the mobile base with the keyboard
    - command of launching simulation:`roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=titanium world:=simple_office_with_people`
      - world:=以降の部分を変更するとシミュレーション空間が変わると考えられる
      - ここではオフィス空間
2. Moving the base throngh velocity commands
    - command of running key_teleop:`rosrun key_teleop key_teleop.py`
      - 矢印キーで操作
      - 押し続けないと止まる

3. Joint Trajectory Contoroller  
    1. Available Interfaces
        - Package:`joint_trajectory_controller`でTiagoの腕を動かすのに使う必要がある
        - trajectoryを送るためにはActionを用いる方法とTopicを用いるものがあるが、どちらもtrajectoryを記述するのにmessage type:`trajectory_msgs/JointTrajectory`を用いる
        - Tiagoのインターフェースは以下の3つが使える
          1. Torso controller
              - 胴体部分(角柱)の1関節をの上下をコントロールするのに使う
              - Topic interface
                  - topic:`/torso_controller/command`, massage:`trajectory_msgs/JointTrajectory`
              - Action interface
                  - `/torso_controller/follow_joint_trajectory`, msg:`control_msgs/FollowJointTrajectoryAction`
          2. Head controller
              - 頭の2関節をコントロールする
              - Topic interface
                  - topic:`head_controller/command`, massage:`trajectory_msgs/JointTrajectory`
              - Action interface
                  - `/head_controller/follow_joint_trajectory`, massage:`control_msgs_/Follow_joint_trajectory`
          3. Arm controller
              - 腕の7関節をコントロールする
              - Topic interface
                  - topic:`arm_controller/command`, massage:`trajectory_msgs/JointTrajectory`
              - Action interface
                  - `/arm_controller/follow_joint_trajectory`, massage:`control_msgs_/Follow_joint_trajectory`
          4. Hey5 controller
              - 研究で使わない
              - Tiago titanium versionで手をコントロールする(手の3つのモーターの制御を行う)
              - Topic interface
                  - topic:`hand_controller/command`, massage:`trajectory_msgs/JointTrajectory`
              - Action interface
                  - `/hand_controller/follow_joint_trajectory`, massage:`control_msgs_/Follow_joint_trajectory`
          5. Gripper controller
              - Tiago steel versionで手のグリッパーをコントロールする
              - Topic interface(並行したグリッパーの仮想関節、つまりグリッパーの指の分離を制御する)
                  - topic:`parallel_gripper_controller/command`, massage:`trajectory_msgs/JointTrajectory`
              - Action interface(グリッパーの指の分離を制御する)
                  - `/parallel_gripper_controller/follow_joint_trajectory`, massage:`control_msgs_/Follow_joint_trajectory`
              - Topic interface(グリッパーの2つのモーターを制御する、つまりそれぞれの指を別々に制御する)   
    2. Execution  
        1. Launching the simulation
            - command:`roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=titanium world:empty`
        2. Launching the node
            - kinematicな設定にアクセスするwaypointを実行するノードは、`run_traj_control`(package:`tiago_trajectory_controller`)
        3. Inspecting the node

4. Moving individual joints  
    - 個々の関節を動かす方法は以下の二つ
        - rqt(Qt GUIベースのGUIソフト開発ツール)から行うもの
        - コマンドライン
    1. Launching the simulation
        - command:`roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=titanium world:=simple_office_with_people`
            - titaniumで人のいるオフィス環境
    2. Moving joints with the rqt GUI
        - command:`rosrun rqt_joint_trajectory_controller rqt_joint_trajectory_controller`
            - これを実行するとrqt GUIのウィンドウが表示される
            - `controller manager ns`:`/controller_maneger`を選択
            - `controller`:動かしたいcontrollerを選択(arm_controller, hand_controller, head_controller, torso_controllerなど, 例ではarm_controller)
            - 実行するためにはまず赤いボタンを押して緑にする
                - これはGUIがtopic:`joint_trajectory_controller`にコマンドを送る準備が出来ていることを示す
            - 各関節の値を変化させて動かせる
            - `speed scaling`で関節の動く速度を調整できる
            - 値を変更した直後に状態が反映される
    3. Moving joints from command line
        - package:`play_motion`にはnode:`move_joint`という各関節を操作できるノードが存在
        - 例えば頭を下げさせるときには以下のコマンドを実行しその下のoutputが返ってきてから頭が動く
            - command:`rosrun play_motion move_joint head_2_joint -0.6 2.0`
            - output:`Moving joint head_2_joint to position -0.6 in 2.0s`
        - 書き方は以下
            - command:`rosrun play_motion move_joint _joint_name_ _joint_position_ _time_`
                - `_time_`は与えられた位置に到達するまでの時間

5. Head control
    - Tiagoに指定の方向に頭を向かせるためのActionの使い方を記述
    - action:`head action`を使用
    1. Launching the simulation
        - command:`roslaunch tiago_gazebo tiago_gazebo.launch robot:=titanium public_sim:=true world:=look_to_point`
    2. Running the example of C++ action client
        - ここではC++で書かれたnodeを用いる
        - node file:`tiago_public_ws/src/tiago_tutorials/look_to_point/src/look_to_point.cpp`
        - topic:`/xtion/rgb/camera_info`, `/xtion/rgb/image_raw`を購読している。
            - `/xtion/rgb/camera_info`:搭載されてるカメラのパラメータが入っている
            - `/xtion/rgb/image_raw`:Tiagoの頭に付いてるRGBDカメラのRGB画像がある
            - 尚、自分でcommand:`rosnode info look_to_point`で確認してもこのxtion系統で購読しているのは`/xtion/rgb/image_raw/compressed`だけだった(なぜ？)
        - ノードを実行すると現れるウィンドウ内のどこかのピクセルを選択するとそこが中心となるように頭が動く 

6. Playing pre-defined upper body motions
    - package:`play_motion`を使用(これはaction serverでもある)
        - 事前に定義されている、同時にグルーピングした複数の関節を動かすことができる
    1. Launching the simulation
        - command:`roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=titanium world:=empty`
            - シミュレーションが立ち上がると同時に`play_motion`のaction serverが自動で立ち上がる
            - `play_motion`内で事前に定義されている動きはfile:`tiago_public_ws/src/tiago_robot/tiago_bringup/config/tiago_motions_titanium.yaml`に記述されており、それがROSパラメータサーバーにロードされる
                - 自分の環境で確認すると`tiago_motions_titanium.yaml`は存在せず、`tiago_public_ws/src/tiago_robot/tiago_bringup/config/motions/`にあるファイル群に定義されてそう(なぜ？)
                - 定義されているmotionの確認には以下のコマンドを用いる
                - command:`rosparam list | grep  "play_motion/motions" | grep "meta/name" | cut -d '/' -f 4`
                    - このチュートリアルでチュートリアルで示されているのはtitaniumのpublic simulationで利用可能なものなのでsteelはもっと減りそう
                - `wave`(一例)
                    - 使用されるパラメータは以下の2つ
                        - `/play_motion/motions/wave/joints`
                        -  `/play_motion/motions/wave/points`
                    - 腕の関節群を使用するmotionを実行するために記述するものを以下のコマンドで確認できる
                        - command:`rosparam get /play_motion/motions/wave/joints`
                    - joint trajectoryつまりmotionを構成する時間経過における一連の座標は以下のcommandで取り出せる(事前に設定された座標)
                        - command:`rosparam get /play_motion/motions/wave/points`
                        - ここで取り出されるのは、各関節の座標(arm_1_joint, ..., arm_7_joint)と開始時刻に対する到達時刻の2つで1列ごとにmotion内でのpositionが異なる
    2. Playing motions
        - グラフィカルなaction clientは以下のコマンドで立ち上げる
            - command:`rosrun actionlib axclient.py /play_motion`
            - 実行するとウィンドウが現れる
                - `Goal`フィールド
                    - `motion_name`:必ず記入しないといけない
                        - デフォルトであるやつを動かすなら変えるのはここだけでOK
                        - 1つ目に設定したmotionは必ず成功するが、連続して設定した別のmotionによってはなぜか`Goal finished with status : ABORTED`で終了するものもある
                        - ちゃんと見たいなら一回シミュレーションごと落とした方がいいかも
                    - `skip_planning`:デフォルトで`False`
                        - joint trajectoryの最初のpositionへの遷移が衝突を防ぐようなplanningを使用しないか否かを設定する
                        - 2つ目以降のpositionへの遷移はプログラマーが安全なトラジェクトリを設定する必要がある、つまりそのようなplanningは使用されない
                        - `True`にするときは現在のkinematic configから最初のpositionまでの遷移が確実に安全になるように設定した時だけにすること
                - `SEND GOAL`ボタンを押すことでaction server:`/play_motion`にリクエストが送られてmotionが実行される
                - 各モーションの映像は記載されてるYoutube動画で全部見れる
    3. C++ and Python action client
        - `tiago_public_ws/src/tiago_tutorials/run_motion`に`play_motion`のaction clientが存在
        - C++
            - command:`rosrun play_motion run_motion _motion_name_`
                - `_motion_name_`の所を変更して使用
                - もし`_motion_name_`を空で渡すと実行可能なmotionのリストが返ってくる
        - Python
            - command:`rosrun play_motion run_motion_python_node.py _motion_name_`
                - `_motion_name_`の所を変更して使用
---

## 3. Autonomous navigation
1. Create a map with gmapping
   - package:`gmapping`を使用してlaser mapの作成を行う
   - mapはlaser scanとmapとの整合性を合わせ、地図内のロボットの信頼できる姿勢の推定値を提供するために、後にAMCLに基づくLocalizationを使用する必要がある(ref: 2. Localization and path planning)
   - このmapはautonomous localizationやpath planningなどに使用することができる
   1. Execution
       1. Generate map
           - command:`roslaunch tiago_2dnav_gazebo tiago_mapping.launch public_sim:=true`
               - gazeboとrvizが立ち上がる
               - 移動させていくとmapが生成されていく
           - command:`rosrun key_teleop key_teleop.py`
               - keyboard teleop用（矢印で操作）
       2. Save map
           - command:`rosservice call /pal_map_manager/save_map "directory: ''"`
           serviceは以下のディレクトリにmapを保存
               - directory:`/home/_use_name_/.pal/tiago_maps/config`
           
           
2. Localization and path planning
    - 以前にlaser scanから作成されたmapを元に、laserとRGBDカメラを障害物を避けるために考慮した上でautonomous navigationをTiagoにさせる
    1. Execution
        - command:`roslaunch tiago_2dnav_gazebo tiago_navigation.launch public_sim:=true lost:=true`
            - rvizとgazeboが立ち上がる
            - シミュレーションだけでなく、autonomous navigationに必要なノードも立ち上がる
            - mapは以前に作成されたものが自動で使用される
            - rviz上に表示されているものは以下になる
                - Particle cloud  
                    - ロボットの位置+方向を、AMCL(パーティクルフィルタ)のパーティクル(小さな赤の矢印)で表示している  
                    - それが集中しているところが自己位置推定において高い確率を持っているところ
                - Global costmap
                    - 障害物の周囲の色のついた領域
                    - map作成時に登録された障害物(static obstacles)に近づきすぎることなくmap上の移動を計算するglobal plannerに使用される
                - Local costmap
                    - Global costmapと似ているが、範囲が小さく、Tiagoの周りにあるため移動する
                    - 以前に作成したmapには存在しなかった新たな特徴(恐らくdynamic obstacles)を考慮するのに使用される
                    - global plannerがグローバルの移動経路を追従する一方で、障害物(static, dynamicの両方)を避けるために、local plannerに使用される
                - Laser scan
                    - laser scanで得られた地点をダークブルーの線で表示している
                    - Global costmap, Local costmapの両方で障害物を追加、削除するのに使用される
                - RGBD scan
                    - マゼンタ色の線で表示されているのが、Tiagoの頭のRGBDカメラから得られた結果をもとに再構成された地点を表している
                    - Lase scan同様Global costmap, Local costmapの両方で障害物を追加、削除するのに使用される
                    - laser scanで捉えきれない(laserが取り付けられている高さより低いor高いため)3Dの障害物の情報を得るのに役立つ
    2. Localization
        - 起動時にはLocalizationが実際の位置と乖離している
        - Localizationは以下のコマンドを実行する
            - command:`rosservice call /global_localization "{}"`
                - (必要?)
                - これによりAMCLの確率的Localizationが起動し、rvizのmap上で大量の赤の矢印が現れる
            - command:`rosrun key_teleop key_teleop.py`
                - これはTiagoを動かして推定値を絞り込むために実行する
                - 回転でもOK、移動によって少しづつ赤色の矢印が一点に集まっていく
            - command:`rosservice call /move_base/clear_costmaps "{}"`
                - Tiagoの悪いLocalizationのせいでcostmapにはエラーがあるため、costmapを綺麗にする
                - これによって障害物(static obstacles)のみを考慮したcostmapになる
                - 以上でnavigationを行う準備が完了する
    3. Autonomous navigation with rviz
        - rviz内の`2D Nav Goal`を使用
        - `key_teleop`の方が優先順位が高いため、そっちが起動していると`2D Nav Goal`は使用できないので注意
        - map内の任意の地点をクリック、ホールドするとマゼンタ色の矢印が現れて向きを変えられ、これで指定する位置とそこでの向きを指定する
        - ホールドを止めると、global plannerが障害物に近づきすぎないように経路を計算し、その結果として青の線(経路)とその先に赤矢印(最終位置での方向)が表示される
        - ホールドを止めた時点でTiagoが移動し始め、最終位置がユーザーが事前指定した許容範囲内にあるときに停止する
---

## 4. MoveIt!
1. Planning in joint space
    - Tiagoの胴体と腕の関節群を関節の可動限界回避と自己衝突回避を保証する目的の関節空間配置にするためにMoveIt!を用いる  
    - MoveIt!は、ロボット制御用のプランニングフレームワークで以下の機能を提供している
        - モーションプランニング
        - マニピュレーション
        - 運動学(順運動学/逆運動学)
        - ナビゲーション
        - 3D環境認識
        > Reference
        > - [ROS入門(35)-MoveItのセットアップ](https://note.com/npaka/n/n90b8db239b38) 
        > - [ROS入門(59)-MoveItのコンセプト](https://note.com/npaka/n/nef3aa7de78c2) 
        > - [MoveIT!の使い方を徹底解説！ダウンロード方法やチュートリアルは？Setup Assistantも利用してみよう！](https://agency-star.co.jp/column/moveit)
    1. Launching simulation
        - command:`roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=steel`
    2. Launching the nodes
        - kinematicな設定から慎重に移動計画が計算されるが、この際に以下のnodeを用いる
            - node:`plan_arm_torso_fk included`(c++), package:`/tiago_public_ws/src/tiago_tutorials/tiago_moveit_tutorial`
            - command:`rosrun tiago_moveit_tutorial plan_arm_torso_fk _torso_joint_vale_ _arm_1_joint_value_ _arm_2_joint_value_ _arm_3_joint_value_ _arm_4_joint_value_ _arm_5_joint_value_ _arm_6_joint_value_ _arm_7_joint_value_`
    3. Inspecting the code
        - 実装における重要な点
            - 関節群の選択
            - plannerの選択
            - 初期状態、最終状態の選択
            - planを見つけるための時間を与える
            - planがあれば実行する,なければ実行しない
            - 関節群のコントローラーへの指示は以下ののActionインターフェースを用いて行う
                - topic:`/arm_controller/follow_joint_trajectory/goal`
                - topic:`/torso_controller/follow_joint_trajectory/goal`
            
2. Planning in cartesian space
    - end-effectorを直交座標系における所定の位置に移動させる
    1. Launching the simulation
        - command:`roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=steel`
    2. Launching the nodes
        - Tiagoのend-effectorを中心とした座標系:`arm_tool_link`を以下のように`/base_footprint`に対する直交座標系の座標を指定して移動させることができる
        - 安全に所定の座標まで移動させるために、package:`tiago_moveit_tutorial`内のnode:`plan_arm_torso_ik`を以下のように呼び出す必要がある
            - command:`rosrun tiago_moveit_tutorial plan_arm_torso_ik _x_coordinate_ _y_coordinate_ _z_coordinate_ roll_angle_ _pitch_angle_ _yaw_angle_`
    3. Inspecting the code(C++)
        - 以下に重要な点を示す
            - 関節群の選択  
            code:  
            `41   moveit::planning_interface::MoveGroup group_arm_torso("arm_torso");`
            - plannerの選択と基準とする座標軸の選択(今回なら`/base_footprint`)し、所定の`/arm_tool_link`の姿勢はmsg:`geometry_msgs/PoseStamped`を使用して設定   
            code:  
            `28   geometry_msgs::PoseStamped goal_pose;`    
            `29   goal_pose.header.frame_id = "base_footprint";`    
            `30   goal_pose.pose.position.x = atof(argv[1]);`    
            `31   goal_pose.pose.position.y = atof(argv[2]);`    
            `32   goal_pose.pose.position.z = atof(argv[3]);`    
            `33   goal_pose.pose.orientation = tf::createQuaternionMsgFromRollPitchYaw(atof(argv[4]), atof(argv[5]), atof(argv[6]));`
            - いつも通りplanを見つけるまでの時間を設定  
            code:  
            `57   group_arm_torso.setPlanningTime(5.0);`
            - もし見つかった場合には実行する  
            code:  
            `68   group_arm_torso.move();`
        - 制御コマンドはarm, torsoそれぞれの制御コントローラに以下のActionインタフェースを使用して送られている
             -  `/arm_controller/follow_joint_trajectory/goal`  
             -  `/torso_controller/follow_joint_trajectory/goal`
             
             
3. Planning in cartesian space with TRACK-IK
    - MoveIt!でTRAC-IKを使用してend-effectorを所定の座標に移動させる
    1. Install TRAC-IK plugin
        - TRAC-IK pluginを使用するために、以下のコマンドを実行する
            - command:`sudo apt-get install ros-$ROS_DISTRO-trac-ik-kinematics-plugin`
            - このコマンドで`trac-ik-kinematics-plugin`, `trac_ik_lib`がworkplaceに追加される
    2. Launching the simulation
         - 2つのコンソールを立ち上げておく
         - 1つ目のコンソールでシミュレーションを起動
             - command:`IK_SOLVER=trac_ik roslaunch tiago_gazebo tiago_gazebo.launch public_sim:=true robot:=steel`
                 - `IK_SOLVER=trac_ik`の部分で、TRAC-IKソルバを指定しており、これがないとKDL solverが選択される
                 - 設定ファイルは、`tiago_moveit_config/config/kinematics_trac_ik.yaml`内にあり、その詳細は[ここ](https://bitbucket.org/traclabs/trac_ik/src/HEAD/trac_ik_kinematics_plugin/)を参照
    3. Launching the nodes
        - コマンドはtutorial4.2のものと同一
        - command:`rosrun tiago_moveit_tutorial plan_arm_torso_ik _x_coordinate_ _y_coordinate_ _z_coordinate_ roll_angle_ _pitch_angle_ _yaw_angle_`
        - 最終姿勢はtutorial4.2と同じであるが、各関節の位置や向きは異なり、これはplannerがMoveIt!依存であり、かつMoveIt!内で異なるIKソルバを使用しているためである
4. Planning with Octomap demo
5. Pick & Place demo